In [4]:
import spacy
import cairosvg
from pathlib import Path
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

sentences = {
    "s3": "I saw a man with a briefcase.",
    "s4": "I saw the planet with a telescope.",
}

out_dir = Path("Illustrations")
out_dir.mkdir(exist_ok=True)

# displaCy style (good contrast)
opts = {"compact": False, "distance": 120, "bg": "#ffffff", "color": "#000000", "font": "Arial"}

for name, text in sentences.items():
    doc = nlp(text)

    # Render SVG from spaCy
    svg = displacy.render(doc, style="dep", options=opts, jupyter=False)

    # Convert to PNG with an explicit white background (prevents transparency)
    png_path = out_dir / f"{name}.png"
    cairosvg.svg2png(
        bytestring=svg.encode("utf-8"),
        write_to=str(png_path),
        output_width=1600,
        background_color="white",   # <- this is the key line
    )

In [ ]:
# Simple script testing for prepositional phrase attachment ambiguity

import spacy

nlp = spacy.load("en_core_web_sm")

def is_ambiguous_pp(sent: str, preps={"with","on","in","at","by","for","from","to","about","over","under","after","before"}):
    doc = nlp(sent)
    for p in doc:
        if p.pos_ == "ADP" and p.text.lower() in preps and any(t.dep_ == "pobj" for t in p.subtree):
            left = [t for t in doc[:p.i] if t.pos_ in ("VERB","AUX","NOUN","PROPN","PRON")]
            if not left:
                continue
            noun = next((t for t in reversed(left) if t.pos_ in ("NOUN","PROPN","PRON")), None)
            verb = next((t for t in reversed(left) if t.pos_ in ("VERB","AUX")), None)
            if noun and verb and noun.i != verb.i:
                return True
    return False

s = input("Enter one sentence: ").strip()
print(f"Input: {s}")
print("This sentence is ambiguous" if is_ambiguous_pp(s) else "This sentence is not ambiguous")

**Disclaimer:** This is a heuristic indicator of possible PP-attachment ambiguity based on spaCy’s parse. Different parsers or contexts may yield different attachments, and the script may return false positives/negatives.